## Back Test Data Generation

In [7]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import warnings
import yaml
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pytz
warnings.filterwarnings('ignore')

### Settings

In [8]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    k_number = cfg['knn']['k_number']
    metric = cfg['knn']['metric']
    algorithm = cfg['knn']['algorithm']
    feature_1 = cfg['knn']['feature_1']
    feature_2 = cfg['knn']['feature_2']
    feature_3 = cfg['knn']['feature_3']
    feature_7 = cfg['knn']['feature_7']
    feature_8 = cfg['knn']['feature_8']
    feature_15 = cfg['knn']['feature_15']
    volume = cfg['feature']['volume']
    volume_size = cfg['sample']['volume_size']
    sample_count = cfg['sample']['count']
    candles = cfg['recommendation']['candle_count']
    pair = cfg['currency']['pair']
    instrument = cfg['currency']['instrument']

In [9]:
print('K Number:',k_number)
print('Metric:', metric)
print('Algorithm:', algorithm)
print('Candle Volume Size:', volume_size)
print('Random Sample Count:', sample_count)
print('Future Candle Count:', candles)
print('Pair:', pair)
print('Instrument:', instrument)

K Number: 5
Metric: euclidean
Algorithm: brute
Candle Volume Size: 5500
Random Sample Count: 4000
Future Candle Count: 7
Pair: EURUSD
Instrument: EUR_USD


In [10]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [11]:
def find_k_similar_candles(candle_id, dataset, k = k_number):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = metric, algorithm = algorithm) 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]][feature_1],
                            dataset.iloc[indices.flatten()[i]][feature_2],
                            dataset.iloc[indices.flatten()[i]][feature_3],
                            dataset.iloc[indices.flatten()[i]][feature_15],
#                            dataset.iloc[indices.flatten()[i]][feature_8],                            
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      feature_1,
                      feature_2,
                      feature_3,
                      feature_15,
#                      feature_8,
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [12]:
filename = '{}_H4.csv'.format(instrument)
data = pd.read_csv(filename)

In [13]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_10',
       'SMA_20', 'F_SMA_5', 'F_SMA_10', 'F_SMA_20', 'O-H', 'O-L', 'O-C', 'H-L',
       'H-C', 'L-C', 'Direction', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5',
       'Trend'],
      dtype='object')

In [14]:
data.shape

(9639, 31)

In [15]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
0,2015-12-09,06:00:00,6,343,2,Wednesday,2015-12-09T06:00:00.000000000Z,6692,1.09079,1.09375,...,0.00311,0.00065,-0.00246,1,-0.00231,0.00065,0.00015,-0.00088,-0.00070,0.0
1,2015-12-09,10:00:00,10,343,2,Wednesday,2015-12-09T10:00:00.000000000Z,6608,1.09307,1.09760,...,0.00598,0.00045,-0.00553,1,-0.00408,0.00045,0.00145,-0.00231,-0.00088,0.0
2,2015-12-09,14:00:00,14,343,2,Wednesday,2015-12-09T14:00:00.000000000Z,13223,1.09712,1.10047,...,0.00543,0.00021,-0.00522,1,-0.00314,0.00021,0.00208,-0.00408,-0.00231,0.0
3,2015-12-09,18:00:00,18,343,2,Wednesday,2015-12-09T18:00:00.000000000Z,11644,1.10026,1.10431,...,0.00453,0.00179,-0.00274,1,-0.00226,0.00179,0.00048,-0.00314,-0.00408,0.0
4,2015-12-09,22:00:00,22,343,2,Wednesday,2015-12-09T22:00:00.000000000Z,3050,1.10246,1.10246,...,0.00181,0.00176,-0.00005,0,0.00176,0.00176,0.00181,-0.00226,-0.00314,0.0


In [16]:
data.describe()

,f_time,julian_date,Weekday,Volume,Open,High,Low,Close,SMA_5,SMA_10,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
count,9639.00000,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000,9639.00000,9639.000000,9639.000000,9639.000000,...,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000,9639.000000
mean,11.36788,181.311962,2.069094,9036.516962,1.143018,1.144468,1.14160,1.143026,1.143017,1.143005,...,0.002868,0.001441,-0.001427,0.505447,-0.000008,0.001441,0.001419,-0.000008,-0.000008,0.510011
std,6.84964,107.043675,1.545465,9333.370690,0.045355,0.045325,0.04535,0.045351,0.045320,0.045291,...,0.001988,0.001512,0.001461,0.499996,0.002069,0.001512,0.001558,0.002069,0.002068,56.310740
min,1.00000,1.000000,0.000000,1.000000,1.036950,1.039150,1.03406,1.036980,1.038700,1.039500,...,0.000000,0.000000,-0.028350,0.000000,-0.027380,0.000000,0.000000,-0.027380,-0.027380,-99.000000
25%,5.00000,87.000000,1.000000,2887.500000,1.111485,1.112880,1.11001,1.111530,1.111400,1.111400,...,0.001540,0.000490,-0.001890,0.000000,-0.000970,0.000490,0.000430,-0.000970,-0.000970,-50.000000
50%,13.00000,181.000000,2.000000,5660.000000,1.135170,1.136640,1.13372,1.135160,1.135000,1.134900,...,0.002380,0.001000,-0.001010,1.000000,-0.000020,0.001000,0.000950,-0.000020,-0.000020,0.000000
75%,17.00000,274.000000,3.000000,11948.500000,1.178715,1.180080,1.17729,1.178710,1.178700,1.178500,...,0.003625,0.001890,-0.000480,1.000000,0.000910,0.001890,0.001890,0.000910,0.000910,52.000000
max,22.00000,366.000000,6.000000,107306.000000,1.254930,1.255600,1.25234,1.254940,1.251400,1.249100,...,0.038520,0.034860,0.000000,1.000000,0.032810,0.034860,0.036470,0.032810,0.032810,99.000000


In [17]:
volume_med = data['Volume'].median()
volume_med

5660.0

## Selecting n random candles where their volume is more than 5500

In [18]:
print('Candle Volume Size:', volume_size)
print('Random Sample Count:', sample_count)

Candle Volume Size: 5500
Random Sample Count: 4000


In [19]:
random_samples = data[data[volume] > volume_size].sample(n = sample_count)

In [20]:
#Random_Candles = np.random.randint(low=1, high=len(data)-40, size=1000)
Random_Candles = list(random_samples.index.values)

In [21]:
#show the fisrt 10 random generated candle numbers
Random_Candles[0:10]

[7684, 5362, 4503, 6941, 4411, 8542, 7975, 7971, 7274, 8621]

# <font color='red'>CANDLE LOOP</font>

In [22]:
CST = pytz.timezone('America/Chicago')
datetime_cst = datetime.now(CST)
print("Date & Time in CST : ", 
      datetime_cst.strftime('%Y:%m:%d %H:%M:%S %Z %z'))

Date & Time in CST :  2022:02:21 19:10:25 CST -0600


In [23]:
%%time

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              
                              'Rec1_Score':[],
                              'Rec1_HH':[],
                              'Rec1_LL':[],
                              
                              'Rec2_Score':[],
                              'Rec2_HH':[],
                              'Rec2_LL':[],
                              
                              'Rec3_Score':[],
                              'Rec3_HH':[],
                              'Rec3_LL':[],
                              
                              'Rec4_Score':[],
                              'Rec4_HH':[],
                              'Rec4_LL':[],
                             })

for candle_no in Random_Candles:
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+candles]
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market = 1  ## Bullish / Buy ##
    else:
        Current_Market = 0  ## Bearish / Sell ##
    
    data = pd.read_csv(filename)
    data = data[[feature_1,
                 feature_2,
                 feature_3,
                 feature_15,
#                 feature_8,
                ]]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []

    for indice in indices[1:5]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+candles]
        
        HH = data.iloc[0]['Close'] - data['High'].max()
        LL = data.iloc[0]['Close'] - data['Low'].min()
#        print("Close:", data.iloc[0]['Close'] , "High: ", data['High'].max(), 'Low: ', data['Low'].min())

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade = 1    ## Buy ##
        else:
            Predicted_Trade = -1   ## Sell ##
        
        predicted_output.append([int(Predicted_Market_Fit) * Predicted_Trade, HH.round(4), LL.round(4)])
        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              
              'Rec1_Score': predicted_output[0][0],
              'Rec1_HH': predicted_output[0][1],
              'Rec1_LL': predicted_output[0][2],
              
              'Rec2_Score': predicted_output[1][0],
              'Rec2_HH': predicted_output[1][1],
              'Rec2_LL': predicted_output[1][2],
              
              'Rec3_Score': predicted_output[2][0],
              'Rec3_HH': predicted_output[2][1],
              'Rec3_LL': predicted_output[2][2],
              
              'Rec4_Score': predicted_output[3][0],
              'Rec4_HH': predicted_output[3][1],
              'Rec4_LL': predicted_output[3][2],
             }
    
    result_output = result_output.append(result, ignore_index = True)

CPU times: user 2h 52min 24s, sys: 4h 22min 58s, total: 7h 15min 23s
Wall time: 23min 37s


In [24]:
now = datetime.now()
today = now.strftime("%d-%m-%Y_%I-%M_%p")

In [25]:
result_output.to_csv('01_Back_Test_Data.csv', header = True, index = False)
result_output.to_csv(today + "_" + "Back_Test_Data_" + pair + '.csv', header = True, index = False)
result_output = pd.read_csv(today + "_" + "Back_Test_Data_" + pair + '.csv')

In [26]:
result_output.head(10)

,Candle_No,Current_Market_Fit,Current_Market,Rec1_Score,Rec1_HH,Rec1_LL,Rec2_Score,Rec2_HH,Rec2_LL,Rec3_Score,Rec3_HH,Rec3_LL,Rec4_Score,Rec4_HH,Rec4_LL
0,7684.0,79.0,0.0,15.0,-0.0024,0.0042,-31.0,-0.0029,0.0029,87.0,-0.0069,0.0013,84.0,-0.0049,0.0032
1,5362.0,67.0,1.0,89.0,-0.0111,0.0049,27.0,-0.0068,0.0035,0.0,-0.0040,0.0033,-92.0,-0.0013,0.0108
2,4503.0,61.0,1.0,-38.0,-0.0016,0.0088,0.0,-0.0039,0.0022,-2.0,-0.0011,0.0016,9.0,-0.0057,0.0063
3,6941.0,53.0,1.0,66.0,-0.0055,0.0032,79.0,-0.0094,0.0053,97.0,-0.0116,0.0023,-70.0,-0.0013,0.0072
4,4411.0,85.0,1.0,45.0,-0.0046,0.0026,61.0,-0.0095,0.0103,71.0,-0.0022,0.0030,-46.0,-0.0004,0.0076
5,8542.0,18.0,0.0,28.0,-0.0032,0.0010,-47.0,-0.0003,0.0077,5.0,-0.0011,0.0025,-66.0,-0.0023,0.0063
6,7975.0,25.0,1.0,37.0,-0.0072,0.0017,-85.0,-0.0009,0.0088,45.0,-0.0040,0.0027,77.0,-0.0039,0.0014
7,7971.0,55.0,0.0,37.0,-0.0069,0.0001,-25.0,-0.0030,0.0043,1.0,-0.0069,0.0054,-5.0,-0.0028,0.0058
8,7274.0,88.0,1.0,63.0,-0.0085,0.0029,57.0,-0.0045,0.0020,52.0,-0.0037,0.0015,-71.0,-0.0006,0.0046
9,8621.0,81.0,0.0,-8.0,-0.0010,0.0054,84.0,-0.0082,0.0007,50.0,-0.0049,0.0011,0.0,-0.0027,0.0062


In [27]:
result_output.shape

(4000, 15)

#### Generating Log File

In [28]:
file = open(today + "_" + "data_generation_log_" + pair + '.txt', "w")
file.write ("Date: " + today + "\n" + \
            "Currency Pair: " + pair + "\n" + \
            "K_Number: " + str(k_number) + "\n" + \
            "KNN_Metric: " + metric + "\n" + \
            "KNN_Algorithm: " + algorithm + "\n" + \
            "Feature: " + feature_1 + "\n" + \
            "Feature: " + feature_2 + "\n" + \
            "Feature: " + feature_3 + "\n" + \
            "Feature: " + feature_7 + "\n" + \
            "Feature: " + feature_8 + "\n" + \
            "Volume Size: " + str(volume_size) + "\n" + \
            "Sample Count: " + str(sample_count) + "\n" + \
            "Candle Counts: " + str(candles) + "\n"
           )
file.close()